### Compute Total Water Storage from PFCLM outputs  

DTT 10/2022

**PFTools Python**:  
(with daily averages)  
`TWS = SUBstorage + SURFstorage + SWEstorage`

_____

This script takes already postprocessed PFCLM outputs as PFB files and computes the daily, (monthly, and yearly) averages to be saved as PFB files.

Inputs:
- Directory where PFCLM daily averages are and directory where you want to save output (these should be the same)
- Hourly PFB files of **subsurface storage [m^3]**, **surface water storage [m^3],** and CLM **Snow Water Equivalent [mm]** outputs
- Water year and day start/end

Outputs:
- PFB files for daily average Total Water Storage in [m^3]
- _PFB files for monthly average Total Water Storage in [m^3] (NOT IMPLEMENTED YET)_
- PFB files for yearly average Total Water Storage in [m^3]
    
User Notes: 
- Because the daily files are read in here and have already been corrected for the CONUS timezone, no adjustments here are necessary
- Make sure to check units of your input files! (see the Inputs list ^)

DTT Notes (10/26/22):
- Need to add in MONTHLY averages

In [1]:
import numpy as np
from parflow import Run
import sys
from parflow.tools.io import read_pfb,write_pfb
import parflow.tools.hydrology as hydro

In [2]:
#these 3 entries (year, day start and day end) will eventually be argv to the script so that it can be run from bash script
water_year = 1999
day_start = 0 #day_start = 0 is the first day of the water year, Oct 1 (e.g., day_start = 2 starts at hour 49)
day_end = 365 #day_end = 365 is the final day of the water year, Sept 30

# water_year = int(sys.argv[1])
# day_start = int(sys.argv[1])
# day_end = int(sys.argv[1])

# directory to save averages to
#directory_out = f'/glade/scratch/tijerina/CONUS2/spinup_WY2003/averages'
directory_out = '/home/dtt2/CONUS2/analysis_scripts/Taylor_test_outputs'

####### TAYLOR ########

nz = 5 #10
ny = 47 #3256
nx = 45 #4442

dx = 1000
dy = 1000
dz = 5 # 200

# apparently it's good to use high numbers when saving files to speed up reading?
# for write_pfb function
p = 5 #72
q = 5 #48
r = 1


In [16]:
# Initialize arrays for daily, monthly, yearly TWS
tws = np.zeros((ny,nx))
tws_m = np.zeros((ny,nx))
tws_y = np.zeros((ny,nx))

for day in range(day_start+1,day_end+1): 
    
    timestamp_day = str(int(day)).rjust(3, '0')
    print(f'Reading storage from day {timestamp_day}')
    
    
    # Read in Subsurface Storage PFBs (5 layers)
    sub_storage_pfb = read_pfb(f'{directory_out}/SUBstorage.{water_year}.daily.{timestamp_day}.pfb')
    # sum all layers of subsurface storage
    sub_storage_sum = np.sum(sub_storage_pfb, axis = 0)

    
    # Read in Surface Storage PFBs 
    surf_storage_pfb = np.squeeze(read_pfb(f'{directory_out}/SURF_WATstorage.{water_year}.daily.{timestamp_day}.pfb'))

    
    # Read in SWE PFBs
    swe_pfb =  np.squeeze(read_pfb(f'{directory_out}/swe_out.{water_year}.daily.{timestamp_day}.pfb'))
    # SWE is in mm, so convert to m^3 for consistent units and storage factor
    swe_meters = swe_pfb * 1000 * dx*dy 

    
    # Calculate Total Water Storage
    tws = sub_storage_sum + surf_storage_pfb + swe_meters
    # save daily TWS PFB
    write_pfb(f'{directory_out}/TWS.{water_year}.daily.{timestamp_day}.pfb',tws,dx=dx,dy=dy,dz=dz,P=p,Q=q,R=r,dist=False)
    
    tws_sum = np.sum(tws)
    write_pfb(f'{directory_out}/TWS_DOMAIN_TOTAL.{water_year}.daily.{timestamp_day}.pfb',tws,dx=dx,dy=dy,dz=dz,P=p,Q=q,R=r,dist=False)
    # monthly TWS
    
    
    # yearly TWS 
    if (day_end-day_start) == 365:
        tws_y += tws / 365
    else:
        print(f'You do not have a full year of daily files, so yearly average will not be calculated!')
        

# save yearly TWS PFB
if (day_end-day_start) == 365:
    write_pfb(f'{directory_out}/TWS.{water_year}.yearly.pfb',tws_y,dx=dx,dy=dy,dz=dz,P=p,Q=q,R=r,dist=False)


Reading storage from day 001
Reading storage from day 002
Reading storage from day 003
Reading storage from day 004
Reading storage from day 005
Reading storage from day 006
Reading storage from day 007
Reading storage from day 008
Reading storage from day 009
Reading storage from day 010
Reading storage from day 011
Reading storage from day 012
Reading storage from day 013
Reading storage from day 014
Reading storage from day 015
Reading storage from day 016
Reading storage from day 017
Reading storage from day 018
Reading storage from day 019
Reading storage from day 020
Reading storage from day 021
Reading storage from day 022
Reading storage from day 023
Reading storage from day 024
Reading storage from day 025
Reading storage from day 026
Reading storage from day 027
Reading storage from day 028
Reading storage from day 029
Reading storage from day 030
Reading storage from day 031
Reading storage from day 032
Reading storage from day 033
Reading storage from day 034
Reading storag

In [7]:
tws_sum

-9708990107074048.0

In [5]:
tws.shape

(47, 45)

#### Quick check yearly and daily TWS to make sure they are similar magnitudes

In [8]:
tws[30,30]

34289857.551114544

In [9]:
tws_y[30,30]

34293190.83671028

In [24]:
x = np.squeeze(read_pfb(f'{directory_out}/TWS.1999.daily.365.pfb'))
x.shape

-9999000000000.0

In [19]:
np.sum(x)

-9708990107074048.0

In [22]:
sub_storage_pfb.shape 

(5, 47, 45)